In [ ]:
import sqlite3
import requests
from bs4 import BeautifulSoup
import time
import re

def parse_stars(text):
    if not text:
        return 0
    t = text.strip().replace(',', '').lower()
    m = re.match(r'^([\d\.]+)k$', t)
    if m:
        return int(float(m.group(1)) * 1000)
    # テキスト中の最初の数値を抽出
    m2 = re.search(r'[\d,\.]+', t)
    if m2:
        try:
            return int(float(m2.group(0).replace(',', '')))
        except Exception:
            return 0
    return 0

DB = 'google_repos.db'
conn = sqlite3.connect(DB)
cur = conn.cursor()
cur.execute('''
CREATE TABLE IF NOT EXISTS repos (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT UNIQUE,
    language TEXT,
    stars INTEGER
)
''')
conn.commit()

headers = {
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0 Safari/537.36"
}
base_url = "https://github.com/orgs/google/repositories"
all_rows = []

# ページごとに time.sleep(1) を入れて GitHub に負荷をかけない
max_pages = 1400  # 必要に応じて増減
for page in range(1, max_pages + 1):
    page_url = f"{base_url}?page={page}"
    print("取得:", page_url)
    try:
        res = requests.get(page_url, headers=headers, timeout=15)
    except requests.RequestException as e:
        print("リクエストエラー:", e)
        break

    if res.status_code != 200:
        print("ステータスコード:", res.status_code)
        break

    soup = BeautifulSoup(res.text, 'html.parser')

    # 複数の候補セレクタを試す（GitHub UIにより変わる）
    repo_items = soup.select('li.Box-row') or soup.select('li[itemprop="owns"]') or soup.select('div.py-4') or soup.select('div.col-12.d-flex')
    print("このページの候補数:", len(repo_items))
    if not repo_items:
        break

    for repo in repo_items:
        # 名前
        name_tag = repo.select_one("a[href^='/google/']") or repo.select_one("a[itemprop='name codeRepository']") or repo.select_one("h3 a")
        if name_tag:
            name = name_tag.get_text(strip=True) or name_tag.get('href', '').split('/')[-1]
        else:
            # hrefから抽出できる場合
            href = repo.find('a', href=re.compile(r'/google/[^/]+'))
            name = href['href'].split('/')[-1] if href and href.get('href') else "Unknown"

        # 言語
        lang_tag = repo.select_one("span[itemprop='programmingLanguage']") or repo.select_one("span.color-text-secondary .ml-0, span[class*='language']")
        language = lang_tag.get_text(strip=True) if lang_tag else None

        # スター数（リンクのテキストまたは aria-label を参照）
        star_tag = repo.find('a', href=re.compile(r'/stargazers$')) or repo.select_one('a[href$="/stargazers"]')
        star_text = ""
        if star_tag:
            star_text = star_tag.get_text(strip=True)
            if not star_text:
                star_text = star_tag.get('aria-label', '')  # 例: "123 users starred this repository"
        stars = parse_stars(star_text)

        all_rows.append((name, language, stars))

    # ページごとにスリープ（必須）
    time.sleep(1)

if not all_rows:
    print("取得データなし")
else:
    try:
        # name に UNIQUE 制約があるので重複は上書き
        cur.executemany('INSERT OR REPLACE INTO repos (name, language, stars) VALUES (?, ?, ?)', all_rows)
        conn.commit()
        print(f"{len(all_rows)} 件を保存しました。")
    except sqlite3.Error as e:
        print("DB保存エラー:", e)

# 保存データを表示
print("\n-- 保存データ（stars DESC） --")
for row in cur.execute('SELECT name, language, stars FROM repos ORDER BY stars DESC'):
    print(row)

conn.close()

取得: https://github.com/orgs/google/repositories?page=1
このページの候補数: 2
取得: https://github.com/orgs/google/repositories?page=2
このページの候補数: 2
取得: https://github.com/orgs/google/repositories?page=3
このページの候補数: 2
取得: https://github.com/orgs/google/repositories?page=4
このページの候補数: 2
取得: https://github.com/orgs/google/repositories?page=5
このページの候補数: 2
取得: https://github.com/orgs/google/repositories?page=6
このページの候補数: 2
取得: https://github.com/orgs/google/repositories?page=7
このページの候補数: 2
取得: https://github.com/orgs/google/repositories?page=8
このページの候補数: 2
取得: https://github.com/orgs/google/repositories?page=9
このページの候補数: 2
取得: https://github.com/orgs/google/repositories?page=10
このページの候補数: 2
取得: https://github.com/orgs/google/repositories?page=11
このページの候補数: 2
取得: https://github.com/orgs/google/repositories?page=12
このページの候補数: 2
取得: https://github.com/orgs/google/repositories?page=13
このページの候補数: 2
取得: https://github.com/orgs/google/repositories?page=14
このページの候補数: 2
取得: https://github.com/orgs/google/reposito